## Boundary Value Problems

In [1]:
using LinearAlgebra
# using Plots
using PyCall
using PyPlot
using DataFrames
using CSV
using Symbolics
using Integrals
using DifferentialEquations
# plotlyjs()
matplotlib.use("TkAgg")

### Shooting Method
A General BVP problem:<br>
$u^{\prime\prime}(x)=g(u,u^\prime,x)$<br>
With boundary conditions as:<br>
$ u(x_1) = u_1$, $ u(x_2) = u_2$

We find the initial value of $u^\prime(x_1)$ by finding the root of the function $u(x_g)-u(x_2)$ where $u(x_g)$ is calculated by rk4 method using some initial guesses for $u^\prime(x_1)$

In [112]:
function RK4(f::Function,xlim,y₀,h)
    xᵢ = xlim[1]:h:xlim[2]
    # print(y₀,'\t',zeros(length(xᵢ)-1,length(y₀)),'\n')
    yᵢ = [transpose(y₀);zeros(length(xᵢ)-1,length(y₀))]
    # print(yᵢ,'\n')

    for (x,y,i) in zip(xᵢ, eachrow(yᵢ),2:length(xᵢ))
        # print(x,' ',y,'\n')
        
            k₁ = h*f(x,y)
            k₂ = h*f(x+h/2,y+k₁/2)
            k₃ = h*f(x+h/2,y+k₂/2)
            k₄ = h*f(x+h,y+k₃)

            yᵢ[i,:] = y + (k₁+2k₂+2k₃+k₄)/6
    end
    
    return yᵢ
end


RK4 (generic function with 1 method)

In [113]:
function linear_interpolation_root_finding(f::Function,x1,x2,ϵ;verbose=false,kill_time=3e10)
    st = time_ns()
    x3=x2-f(x2)*(x2-x1)/(f(x2)-f(x1))
    i=1
    xᵢ=[]
    while abs(f(x3))>ϵ
        if (time_ns()-st)>kill_time
            println("TIMEOUT! Did not converge for the given precision OR initial guesses")
            return NaN,"DID NOT CONVERGE"
        end
        i=i+1
        x3=x2-f(x2)*(x2-x1)/(f(x2)-f(x1))

        if verbose
            print("Iteration: ",i)
            print(" Current root: ",x3,'\n')
        end
        
        if f(x3)*f(x1)<0
            x2=x3
        else
            x1=x3
        end    

        xᵢ=[xᵢ;x3]
    end
    if verbose
        return x3,xᵢ
    end
    return x3,i
end

linear_interpolation_root_finding (generic function with 1 method)

In [129]:
function shooting_method(f::Function,xlim,y₀,yf,y′lim,h,ϵ;plot=false)
    plt.close("all")
    plt.hlines(yf,xlim[1],xlim[2],linestyle="--")
    plt.xlabel("x")
    plt.ylabel("u")
    plt.title("BVP Solution by Shooting Method\n Dashed blue line is the target")
    function int_func(y′g;plot=plot)
        y=RK4(f,xlim,[y₀,y′g],h)
        if plot
            plt.plot(xlim[1]:h:xlim[2],y[:,1])
            plt.text(xlim[2],y[end,1],"u'=$y′g")
        end
        y[end,1]-yf
    end
    int_func_plot = (g)->int_func(g,plot=plot) 
    sol=linear_interpolation_root_finding(int_func_plot,y′lim[1],y′lim[2],ϵ)
    print("Value of u′(",y₀,"): ",sol[1],"\n")
    print("Number of iterations: ",sol[2],"\n")
    if plot
        plt.show()
        
    end
end

shooting_method (generic function with 2 methods)

#### Linear BVP

In [114]:
function u₂(u,x)
    (1-x/5)*u+x
end

function u₁(u′)
    u′
end

function f(x,y)
    u,u′=y
    [u₁(u′),u₂(u,x)]
end

f (generic function with 1 method)

In [130]:
shooting_method(f,[1,3],2,-1,[-2,2],0.0001,1e-7,plot=true)

Value of u′(2): -3.4949853954695502
Number of iterations: 1


Only a single iteration results in finding the correct solution (after initial guesses at two random points). Since the ODE is linear, this is expected.

### Non Linear BVP

In [123]:
function v₂(v,v′,x)
    (1-x/5)*v*v′+x
end

function v₁(v′)
    v′
end

function g(x,y)
    v,v′=y
    [v₁(v′),v₂(v,v′,x)]
end

g (generic function with 1 method)

In [127]:
shooting_method(g,[1,3],2,-1,[-1.5,-2],0.0001,1e-4,plot=true) 

Value of u′(2): -2.0160524313053663
Number of iterations: 3


Now, it takes three more iterations to reach the correct value